In [1]:
import requests
from bs4 import BeautifulSoup


In [2]:
def get_soup(url, params = None, headers = None):
    response = requests.get(url, params = params,
                            headers = headers)
    soup = BeautifulSoup(response.content, "html.parser")
    return soup

def get_num_pages(url):
    soup = get_soup(url)
    pages_div = soup.find_all("div", class_ = "in-pagination__item")
    pages_a = soup.find_all("a", class_ = "in-pagination__item")
    
    pages_div = [page.text for page in pages_div]
    pages_a = [page.text for page in pages_a]
    
    num_list = list()
    for page in pages_a + pages_div:
        try:
            num = int(page)
            num_list.append(num)
        except:
            continue
    return max(num_list)

In [3]:
def get_info_from_annuncio(annuncio):
    title = annuncio.find("a", class_ = "in-reListCard__title").text
    price = annuncio.find("div", class_ = "in-reListCardPrice").text
    infos = annuncio.find_all("div", class_ = "in-reListCardFeatureList__item")
    infos_text = [info.text for info in infos]
    res = {"title" : title, "price" : price, "infos":infos_text}
    return res

def get_immobiliare_info(url, params = None, headers = None):
    soup = get_soup(url, params= params, headers= headers)
    terreni = soup.find_all("div", class_ = "nd-mediaObject__content")
    res = []
    for i,terreno in enumerate(terreni):
        res.append(get_info_from_annuncio(terreno))
    return res

In [4]:


def get_all_from_regione(regione):

    url = "https://www.immobiliare.it/vendita-terreni/{}/?criterio=rilevanza&superficieMinima=20000&idTipologia=107".format(regione)
    url_with_page_num = url+"&pag={}"

    num_pages = get_num_pages(url)
    items = list()
    for i in range(1,num_pages+1):
        items.extend(get_immobiliare_info(url_with_page_num.format(i)))
    return items

all_annunci = dict()
for regione in ["lombardia", "veneto"]:
    all_annunci[regione]= get_all_from_regione(regione)

In [7]:
for regione in all_annunci:
    print(regione, "number of terreni:", len(all_annunci[regione]))

lombardia number of terreni: 334
veneto number of terreni: 227


In [8]:
import json
with open("annunci.json", 'w') as json_file:
    json.dump(all_annunci, json_file)